## Maximum Elder Facilities Covering Location Problem  solved by Gurobi

In [ ]:
from gurobipy import *
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def gurobi_solver_bclp(users, facilities, demand, PN, radius, w=0.8):
    """
    使用 Gurobi 求解 BCLP (Backup Coverage Location Problem) 模型。
    该实现与 PyTorch BCLP 函数的逻辑一致。

    Args:
        users (np.array): 用户/需求点位置，形状为 [N, 2]。
        facilities (np.array): 候选设施位置，形状为 [M, 2]。
        demand (list or np.array): 每个用户的需求量，长度为 N。
        PN (int): 需要选择的设施总数（等价于 p）。
        radius (float): 设施的覆盖半径。
        w (float): 目标函数中主覆盖和备用覆盖之间的权重。

    Returns:
        tuple: (
            x_result,               # 选中的设施（布尔列表，长度 M）
            y_result,               # 主覆盖（≥1 次）标记（布尔列表，长度 N）
            obj,                    # 目标函数值
            coverage_rate,          # 主覆盖比例（覆盖需求量/总需求量，0–1）
            covered_demand,         # 主覆盖的需求量
            total_demand,           # 总需求量
            backup_coverage_rate,   # 备份覆盖比例（≥2 次覆盖，0–1）
            backup_covered_demand   # 备份覆盖的需求量
        )
    """
    # 1. 问题数据准备
    N = len(users)      # 需求点数量
    M = len(facilities) # 候选设施点数量

    # 计算距离矩阵
    dist_matrix = np.linalg.norm(facilities[:, np.newaxis, :] - users[np.newaxis, :, :], axis=2)

    # 创建二元覆盖矩阵 A (A_ij = 1 如果设施 i 覆盖用户 j)
    coverage_matrix = (dist_matrix <= radius).astype(int)

    # 2. Gurobi 模型初始化
    model = Model('BCLP')
    model.setParam('OutputFlag', False) # 关闭冗余输出

    # 3. 添加决策变量
    x = {}  # x_j: 1 如果设施 j 被选择, 0 否则
    y = {}  # y_i: 1 如果用户 i 被至少一个设施覆盖
    u = {}  # u_i: 1 如果用户 i 被至少两个设施覆盖

    for j in range(M):
        x[j] = model.addVar(vtype=GRB.BINARY, name=f"x({j})")
    for i in range(N):
        y[i] = model.addVar(vtype=GRB.BINARY, name=f"y({i})")
        u[i] = model.addVar(vtype=GRB.BINARY, name=f"u({i})")

    model.update()

    # 4. 设置目标函数
    # Maximize Z = w * sum(demand_i * y_i) + (1-w) * sum(demand_i * u_i)
    primary_coverage_benefit = quicksum(demand[i] * y[i] for i in range(N))
    backup_coverage_benefit = quicksum(demand[i] * u[i] for i in range(N))

    model.setObjective(
        w * primary_coverage_benefit + (1 - w) * backup_coverage_benefit,
        GRB.MAXIMIZE
    )

    # 5. 添加约束条件
    # a. 选择设施的总数必须为 PN
    model.addConstr(quicksum(x[j] for j in range(M)) == PN, "Facility_Selection_Constraint")

    # b. BCLP 核心约束
    for i in range(N):
        # 找到所有能覆盖用户 i 的设施集合 (N_i)
        covering_facilities_indices = np.where(coverage_matrix[:, i] == 1)[0]

        if len(covering_facilities_indices) > 0:
            # 计算覆盖用户i的已选设施总数
            coverage_sum = quicksum(x[j] for j in covering_facilities_indices)

            # 约束 y_i: 如果覆盖总数 >= 1, y_i 可以为 1
            model.addConstr(coverage_sum >= y[i], f"Primary_Coverage_{i}")

            # 约束 u_i: 如果覆盖总数 >= 2, u_i 可以为 1
            # (这个约束也隐含了 u_i <= y_i)
            model.addConstr(coverage_sum >= 2 * u[i], f"Backup_Coverage_{i}")
        else:
            # 如果没有任何设施能覆盖该用户，则其 y_i 和 u_i 必须为0
            model.addConstr(y[i] == 0)
            model.addConstr(u[i] == 0)

    # 6. 求解模型
    model.optimize()

    # 7. 返回结果
    if model.status == GRB.Status.OPTIMAL:
        obj = model.ObjVal
        x_result = [x[j].X > 0.5 for j in range(M)]
        y_result = [y[i].X > 0.5 for i in range(N)]
        u_result = [u[i].X > 0.5 for i in range(N)]

        total_demand = sum(demand)
        covered_demand = sum(demand[i] for i in range(N) if y_result[i])
        coverage_rate = covered_demand / total_demand if total_demand > 0 else 0

        backup_covered_demand = sum(demand[i] for i in range(N) if u_result[i])
        backup_coverage_rate = backup_covered_demand / total_demand if total_demand > 0 else 0

        return (
            x_result, y_result, obj,
            coverage_rate, covered_demand, total_demand,
            backup_coverage_rate, backup_covered_demand
        )
    else:
        # 如果找不到最优解，返回空结果
        total_demand = sum(demand)
        return None, None, 0, 0, 0, total_demand, 0, 0

## Load the real-world datasets

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

### Demand Population Distribution

In [ ]:
%%time
ls = gpd.read_file("data/电网/powertower_tianhe_a/HaizhuPowertower_3857.shp")
ls['point_x_1'] = ls.geometry.x
ls['point_y_1'] = ls.geometry.y
print(ls.head(3))
print(ls.crs)
ls['mianji'] = ls.a# 每个需求点的需求量
total_pop = sum(ls['mianji'])
print("The number of records is ", len(ls))
print("The total speed unit are ", total_pop)

### Candidate Elder Facilities Location

In [ ]:
ef = pd.read_csv("data/电网/powertower_tianhe_a/HaizhuPowertower.csv")
# ef = pd.read_csv("./data/特征注入/候选点.csv")
# sites = np.array(sitedf[['NORM_X', 'NORM_Y']], dtype=np.float64)
print("The number of elder facilities in Seattle area is ", len(ef))
ef.head(3)

## Normalization

In [ ]:
# 归一化函数 通过计算x和y的最大值和最小值，选择xy之间大的尺度S，将数据归一化到[0, 1]区间
def Normalization(x, y):
    # 找到x和y的最大值和最小值
    max_x, max_y = np.max(x), np.max(y)
    min_x, min_y = np.min(x), np.min(y)
    # 计算x和y的范围（空间差值）
    S_x = (max_x-min_x)
    S_y = (max_y-min_y)
    # 取x和y的最大范围作为标准化的尺度
    S = max(S_x, S_y)
    # 将x和y归一化到[0, 1]区间
    new_x, new_y = (x-min_x)/S, (y-min_y)/S
    # 将归一化后的x和y组合成一个二维数组
    data_xy = np.vstack((new_x, new_y))
    Data = data_xy.T
    # 返回归一化后的x和y以及尺度S
    return new_x, new_y, S

In [ ]:
# 提取数据 ls是总的需求点数据 bbs是候选的设施点数据
ls_X = np.array(ls['point_x_1'])
ls_Y = np.array(ls['point_y_1'])
ef_X = np.array(ef['POINT_X'])
ef_Y = np.array(ef['POINT_Y'])
# 合并ls，bbs 的  x，y 坐标 为 X,Y
X = np.concatenate([ls_X, ef_X])
Y = np.concatenate([ls_Y, ef_Y])
# 归一化 X,Y
NORM_X, NORM_Y, S = Normalization(X, Y)
# 将归一化后的数据添加到ls和sitedf中
ls['NORM_X'] = NORM_X[:len(ls)]
ls['NORM_Y'] = NORM_Y[:len(ls)]
ef['NORM_X'] = NORM_X[len(ls):]
ef['NORM_Y'] = NORM_Y[len(ls):]

In [ ]:
ef.head(3)

### Visualization of the input data

In [ ]:
# 底图上绘制比例尺
def render_scale_bar(ax, x, y, segments=2, height=0.01, seg_length=2000, unit='m', linewidth=1.):
    # ax : matplotlib的坐标轴对象 绘制比例尺
    # x, y : 比例尺的左下角坐标
    # segments : 分段数 默认为2
    # height ： 比例尺的高度 默认为0.01
    # seg_length : 每段的长度 默认为2000
    # unit : 单位 默认为'm'
    # linewidth : 线条宽度 默认为1.0

    # 单位转换字典 1 英里 = 1609.34 米
    unit_scale_factor = {
        'm': 1,
        'km': 1000,
        'meters': 1,
        'kilometers': 1000,
        'miles': 1609.34,
        'mi': 1609.34,
        'ft': 0.3,
        }
    # x_lim和y_lim  获取当前图形的坐标轴范围
    x_lim = ax.get_xlim()
    y_lim = ax.get_ylim()

    # how much percent does one unit takes on the x axis
    # 计算x轴和y轴上每个单位长度所占的百分比
    x_per_unit = 1. / (x_lim[1] - x_lim[0])
    y_per_unit = 1. / (y_lim[1] - y_lim[0])

    # base for ticks (0, 1)
    x_base = [x + seg_length * unit_scale_factor[unit] * x_per_unit * i for i in range(0, segments + 1)]
    ax.axhline(y_lim[0] + y / y_per_unit, x_base[0], x_base[-1], c='black')
    y_base = [y, y + height]
    for i in range(segments + 1):
        ax.axvline(x_lim[0] + x_base[i] / x_per_unit, y, y + height, c='black')
        xy = (x_lim[0] + x_base[i] / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit)  # data_coords
        ax.text(xy[0], xy[1], s='{}'.format(int(seg_length * i)), horizontalalignment='center', verticalalignment='center')
    ax.text(x_lim[0] + (x_base[-1] + 0.02) / x_per_unit, y_lim[0] + (y - 0.015) / y_per_unit,
            s=unit, horizontalalignment='left',
            verticalalignment='center')

In [ ]:
# 绘制指北针
def render_north_arrow(ax, x, y, size, ratio = 1):
    # ax： matplotlib的坐标轴对象 绘制指北针
    # x, y： 指北针的左下角坐标
    # size： 指北针的大小
    # ratio： 指北针的比例 默认为1
    path = [(0, 1), (-ratio, -1), (0, -0.5), (ratio, -1), (0, 1)]
    path = [(i[0] * size + x, i[1] * size + y) for i in path]
    arrow = plt.Polygon(path, color='black', transform=ax.transAxes)
    ax.add_patch(arrow)
    ax.text(x, y-size*2, s = 'N', horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

In [ ]:
# 筛选出在ls范围内的sitedf 得到 sitedf1
ef1 = ef[ef['POINT_X'] < max(ls['point_x_1'])]
ef1 = ef1[ef1['POINT_X'] > min(ls['point_x_1'])]
ef1 = ef1[ef1['POINT_Y'] < max(ls['point_y_1'])]
ef1 = ef1[ef1['POINT_Y'] > min(ls['point_y_1'])]

In [ ]:
%%time
# 绘制输入数据的散点图
fig, ax = plt.subplots(figsize=(20,18))
# ax.scatter(ls['POINT_X'],ls['POINT_Y'], color='Blue', s=ls['speed_pct_freeflow_rev'])

# 创建一个自定义的颜色映射
cmap = plt.cm.Blues
new_cmap = colors.ListedColormap(cmap(np.linspace(0.15, 1, 256)))  # 使得低值区域不那么浅
# 绘制LandScan数据的散点图
ls.plot(ax = ax, column=ls['mianji'], k=5, markersize=20, cmap=new_cmap, label = 'over-65 Population')
# 绘制Billboards数据的散点图
ax.scatter(ef1['POINT_X'], ef1['POINT_Y'], c='C1', s=2, label ='Elder Facilities')

ax.axis('scaled')
ax.tick_params(axis='both', left=False, top=False, right=False,
               bottom=False, labelleft=False, labeltop=False,
               labelright=False, labelbottom=False)

ax.set_title("Input Data", fontsize=20)
render_scale_bar(ax = ax, x=0.05, y=0.05)
render_north_arrow(ax = ax, x = 0.95, y = 0.95, size = 0.01, ratio = 0.7)
ax.legend(loc='lower right', markerscale = 1)

In [ ]:
# 根据覆盖范围从给定的点数据选择（自己定义方法，根据方法选择不同的列）
def generate_candidate_sites(sites, M=100, heuristic = None):
    # sites : Pandas dataframe 包含多个站点数据
    # M : 需要生成的候选站点数量
    # heuristic : 选择站点的启发式方法（’coverage', 'coverage_e', 'impression', 'impression_e'）
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        sites: a Pandas DataFrame with X, Y and other characteristic
        M: the number of candidate sites to generate
        heuristic:
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
    # M > sites的长度时，M设置为None ： 返回整个数据集
    if M is not None:
        if M > len(sites):
            M = None
    # 如果没有指定启发式方法，则随机选择M个站点
    if heuristic is None or heuristic == '':
        if M is None:
            return sites
        # 随机选择M个站点
        index = np.random.choice(len(sites), M)
        sites.iloc[index]
        return sites.iloc[index]
    # 根据启发式方法选择站点
    # 若为 'coverage'，则根据pop_covered_2km 进行排序
    elif heuristic == 'coverage':
        sites = sites.sort_values(by='pop_covered_2km', ascending=False).reset_index()
        if M is None:
            return sites
        sites.iloc[:M]
        return sites.iloc[:M]
    # 若为 'coverage_e'，则根据pop_covered_2km_exclusive 进行排序
    elif heuristic == 'coverage_e':
        sites = sites.sort_values(by='pop_covered_2km_exclusive', ascending=False).reset_index()
        if M is None:
            return sites
        sites.iloc[:M]
        return sites.iloc[:M]
    # 若为 'impression'，则根据weeklyImpr列 进行排序
    elif heuristic == 'impression':
        sites = sites.sort_values(by='weeklyImpr', ascending=False).reset_index()
        if M is None:
            return sites
        sites.iloc[:M]
        return sites.iloc[:M]
    # 若为 'impression_e'，则根据weeklyImpr_2km_exclusive 进行排序
    elif heuristic == 'impression_e':
        sites = sites.sort_values(by='weeklyImpr_2km_exclusive', ascending=False).reset_index()
        if M is None:
            return sites
        sites.iloc[:M]
        return sites.iloc[:M]


In [ ]:
# # 给选定的站点绘制结果
# def plot_result(ls,opt_sites,radius):
#     # ls ： LandScan数据
#     # opt_sites : 优化后的站点数据
#     # radius : 站点的服务范围半径
#     '''
#     Plot the result
#     Input:
#         points: input points, Numpy array in shape of [N,2]
#         opt_sites: locations K optimal sites, Pandas DataFrame
#         radius: the radius of circle
#     '''
#     fig, ax = plt.subplots(figsize=(20,15))
#     # 根据ls中的speed_pct_freeflow_rev列绘制LandScan数据的散点图 颜色渐变
#     ls.plot(ax = ax, column=ls['mianji'], k=5, markersize=10, cmap=new_cmap, label = 'over_65 Population')
#     # 绘制优化后的站点数据的散点图的状态
#     legend_plot_flag = {'current':False,'selected':False}
#
#     # 遍历每一个优化后的站点， site 包含站点的位置信息和其他特征
#     for cnt, site in opt_sites.iterrows():
#         # 如果站点有 'current' 列且值为 True，则绘制当前的广告牌  红色 +
#         if 'current' in opt_sites.columns and site['current'] == True:#ncurrent
#             if legend_plot_flag['current'] == False:
#                 ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=1, label = 'Current Billboards')
#                 circlelabel = 'Service Range for Current Billboards'
#                 legend_plot_flag['current'] = True
#             ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=100)
# #           circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='red', fill=False, lw=2, label = circlelabel)
#             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='red', fill=False, lw=2)
#
#             ax.add_artist(circle)
#         # 如果站点没有 'current' 列或值为 False，则绘制优化后的选定的广告牌  C1颜色 +
#         else:
#             if legend_plot_flag['selected'] == False:
#                 ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=1, label = 'Optimized Selected Elder Facilities')
#                 circlelabel = 'Service Range for Optimized Selected Billboards'
#                 legend_plot_flag['selected'] = True
#             ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=100)
# #             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='C1', fill=False, lw=2, label = circlelabel)
# #             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='C1', fill=False, lw=2)
# #             ax.add_artist(circle)
#
#     # 坐标轴设置
#     ax.axis('scaled')
#     ax.tick_params(axis='both',left=False, top=False, right=False,
#                        bottom=False, labelleft=False, labeltop=False,
#                        labelright=False, labelbottom=False)
#     title = 'Selected ' + str(len(opt_sites)) + ' Sites that Serve ' + str(radius) + ' m'
#     ax.set_title(title, fontsize=20)
#     # 绘制比例尺和指北针
#     render_scale_bar(ax = ax, x=0.05, y=0.05)
#     render_north_arrow(ax = ax, x = 0.95, y = 0.95, size = 0.01, ratio = 0.7)
#     # 添加图例
#     ax.legend(loc='lower right', markerscale = 10)
#     return ax

In [ ]:
# 求解
np.random.seed() # 生成随机种子
ef_ = generate_candidate_sites(ef, M=None, heuristic="") # 生成候选站点数据（返回所有站点）
users = np.array(ls[['NORM_X', 'NORM_Y']]) # ls转化为 Numpy 数组
facilities_all = np.array(ef_[['NORM_X', 'NORM_Y']]) # bbs_转化为 Numpy 数组
# demand = np.array(ls['speed_pct_freeflow_rev_norm'])
demand = np.array(ls['mianji']) # 列speed_pct_freeflow_rev 交通流作为需求量

PN = 25 # M = 20
real_radius = 500 # 真实的服务范围半径
radius = real_radius/S # 将真实的服务范围半径归一化到[0, 1]区间

# 计算了所有设施与所有用户之间的欧几里得距离
A = np.sum((facilities_all[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5

# 生成覆盖矩阵 A[i,j] = 1 表示设施点i可以覆盖需求点j，A[i,j] = 0 表示设施点i不能覆盖需求点j
mask1 = A <= radius
A[mask1] = 1
A[~mask1] = 0
print(S)

In [ ]:
# gurobi求解值

start = time.time()

w1 = 0.6  # Weight for coverage
w2 = 0.4  # Weight for distance
da = 1000 / S  # Distance tolerance lower bound (in meters)
db = 2000 / S# Distance tolerance upper bound (in meters)
D_max = 2000 / S   # Maximum distance (in meters)
x_result, y_result, obj , coverage_rate, covered_demand, total_demand, backup_coverage_rate, backup_covered_demand = gurobi_solver_bclp(users, facilities_all, demand, PN, radius, w=0.8)
# x_result 和 y_result 分别表示哪些设施被选择和哪些用户被覆盖，obj 是优化后的目标值（即加权覆盖量）
print(f"The objective value is: {obj}")
print(f"Primary coverage rate: {coverage_rate:.2%}")
print(f"Backup coverage rate:  {backup_coverage_rate:.2%}")
solutions = []
for i in range(len(facilities_all)):
    if x_result[i] == 1.0:
        solutions.append(i)
end = time.time()-start
print(f"The running time of Groubi is: {end}")
print(solutions)
# # 开始循环
# objective_values = []
# execution_times = []
#
# # 定义循环次数
# num_iterations = 1000
# for i in range(num_iterations):
#     start_run_time = time.time()
#
#     # 1. 从421个候选点中随机选择100个
#     candidate_indices = np.random.choice(len(facilities_all), 100, replace=False)
#     facilities = facilities_all[candidate_indices]
#
#     # 2. 为当前选定的100个设施点生成覆盖矩阵A
#     A = np.sum((facilities[:, np.newaxis, :] - users[np.newaxis, :, :]) ** 2, axis=-1) ** 0.5
#     mask1 = A <= radius
#     A[mask1] = 1
#     A[~mask1] = 0
#
#     # 3. 调用Gurobi求解器
#     x_result, y_result, obj, coverage_rate, covered_demand, total_demand = gurobi_solver_EF_MCLP(
#         users, facilities, demand, PN, A, w1, w2, da, db, D_max
#     )
#
#
#     end_run_time = time.time()
#
#     # 4. 记录本次运行的目标值和时间
#     objective_values.append(obj)
#     execution_times.append(end_run_time - start_run_time)
#
#     # (可选) 打印每次迭代的进度
#     print(f"迭代 {i+1}/{num_iterations} 完成: Obj = {obj:.2f}, 时间 = {end_run_time - start_run_time:.4f}s")
#
#
# # --- 循环结束后计算并输出平均值 ---
#
# average_objective = np.mean(objective_values)
# average_time = np.mean(execution_times)
#
# print("\\n" + "="*50)
# print(f"在 {num_iterations} 次随机抽样运行后:")
# print(f"平均目标函数值 (Average obj): {average_objective:.4f}")
# print(f"平均运行时间 (Average time): {average_time:.4f} 秒")
# print("="*50)

In [ ]:
# 给选定的站点绘制结果
def plot_result(ls,opt_sites,radius):
    # ls ： LandScan数据
    # opt_sites : 优化后的站点数据
    # radius : 站点的服务范围半径
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Pandas DataFrame
        radius: the radius of circle
    '''
    fig, ax = plt.subplots(figsize=(20,15))
    # 根据ls中的speed_pct_freeflow_rev列绘制LandScan数据的散点图 颜色渐变
    ls.plot(ax = ax, column=ls['mianji'], k=5, markersize=10, cmap=new_cmap, label = 'over_65 Population')
    # 绘制优化后的站点数据的散点图的状态
    legend_plot_flag = {'current':False,'selected':False}

    # 遍历每一个优化后的站点， site 包含站点的位置信息和其他特征
    for cnt, site in opt_sites.iterrows():
        # 如果站点有 'current' 列且值为 True，则绘制当前的广告牌  红色 +
        if 'current' in opt_sites.columns and site['current'] == True:#ncurrent
            if legend_plot_flag['current'] == False:
                ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=1, label = 'Current Billboards')
                circlelabel = 'Service Range for Current Billboards'
                legend_plot_flag['current'] = True
            ax.scatter(site['POINT_X'], site['POINT_Y'], c='red', marker='+', s=100)
#           circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='red', fill=False, lw=2, label = circlelabel)
            circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='red', fill=False, lw=2)

            ax.add_artist(circle)
        # 如果站点没有 'current' 列或值为 False，则绘制优化后的选定的广告牌  C1颜色 +
        else:
            if legend_plot_flag['selected'] == False:
                ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=1, label = 'Optimized Selected Elder Facilities')
                circlelabel = 'Service Range for Optimized Selected Billboards'
                legend_plot_flag['selected'] = True
            ax.scatter(site['POINT_X'], site['POINT_Y'], c='C1', marker='+', s=100)
#             circle = plt.Circle(site[['POINT_X','POINT_Y']], radius, color='C1', fill=False, lw=2, label = circlelabel)
            circle = plt.Circle(site[['POINT_X','POINT_Y']], radius/2, color='C1', fill=False, lw=2)
            ax.add_artist(circle)

    # 坐标轴设置
    ax.axis('scaled')
    ax.tick_params(axis='both',left=False, top=False, right=False,
                       bottom=False, labelleft=False, labeltop=False,
                       labelright=False, labelbottom=False)
    ax.grid(False)
    title = 'Selected ' + str(len(opt_sites)) + ' Sites that Serve ' + str(radius) + ' m(Gurobi)'
    ax.set_title(title, fontsize=20)
    # 绘制比例尺和指北针
    render_scale_bar(ax = ax, x=0.05, y=0.05)
    render_north_arrow(ax = ax, x = 0.95, y = 0.95, size = 0.01, ratio = 0.7)
    # 添加图例
    ax.legend(loc='lower right', markerscale = 10)
    return ax
opt_sites = ef_.iloc[solutions]  # Get the selected facilities based on the solutions

plot_result(ls,opt_sites,real_radius)

In [ ]:
# 批量参数遍历并导出到 Excel

def run_param_sweep_and_export(users, facilities, demand, radius, w_list, p_list, output_path='solution.xlsx'):
    """
    遍历 w 与 p（设施个数）组合，计算主/备覆盖（比例与点数），写入两个工作表：
      - Sheet1: ['w','num','primary coverage(%)','backup coverage(%)']  # 百分比写 0–1 小数
      - Sheet2: ['w','num','primary coverage','backup coverage']        # 覆盖到的需求点数量
    参数：
      users: np.ndarray [N,2]（坐标需与 radius 同尺度，当前为归一化坐标）
      facilities: np.ndarray [M,2]（归一化坐标）
      demand: 1D 数组，长度 N
      radius: 覆盖半径（与 users/facilities 同尺度）
      w_list: 权重列表
      p_list: 设施数量列表（等价于 num/p/PN）
      output_path: 输出 Excel 路径（建议指向作图本所在目录）
    返回：df_pct, df_cnt 两个 DataFrame
    """
    demand = np.array(demand, dtype=float)
    total_demand = float(np.sum(demand))

    rows_pct = []   # Sheet1（比例 0–1）
    rows_cnt = []   # Sheet2（点数）

    for w in w_list:
        for p in p_list:
            # 调用已实现的 Gurobi 求解器
            x_result, y_result, obj, coverage_rate, covered_demand, total_demand_solver, backup_coverage_rate, backup_covered_demand = \
                gurobi_solver_bclp(users, facilities, demand, p, radius, w=w)

            if x_result is None:
                primary_points = 0
                backup_points = 0
                primary_pct = 0.0
                backup_pct = 0.0
            else:
                sel_idx = [j for j, flag in enumerate(x_result) if flag]
                if len(sel_idx) == 0:
                    primary_points = 0
                    backup_points = 0
                    primary_pct = 0.0
                    backup_pct = 0.0
                else:
                    sel_fac = facilities[sel_idx]
                    # 计算选中设施对所有需求点的覆盖次数
                    dist = np.linalg.norm(sel_fac[:, np.newaxis, :] - users[np.newaxis, :, :], axis=2)
                    cover = (dist <= radius).astype(int)        # [#sel, N]
                    cover_counts = np.sum(cover, axis=0)        # [N]

                    primary_points = int(np.count_nonzero(cover_counts >= 1))
                    backup_points  = int(np.count_nonzero(cover_counts >= 2))

                    primary_demand = float(np.sum(demand[cover_counts >= 1]))
                    backup_demand  = float(np.sum(demand[cover_counts >= 2]))
                    denom = total_demand if total_demand > 0 else 1.0
                    primary_pct = primary_demand / denom
                    backup_pct  = backup_demand  / denom
                    obj_value = obj

            rows_pct.append({'w': float(w), 'num': int(p),
                              'primary coverage(%)': primary_pct,
                              'backup coverage(%)':  backup_pct,
                              'Obj': obj_value})
            rows_cnt.append({'w': float(w), 'num': int(p),
                              'primary coverage': int(primary_points),
                              'backup coverage':  int(backup_points)})

    df_pct = pd.DataFrame(rows_pct)
    df_cnt = pd.DataFrame(rows_cnt)

    with pd.ExcelWriter(output_path) as writer:
        df_pct.to_excel(writer, index=False, sheet_name='Sheet1')
        df_cnt.to_excel(writer, index=False, sheet_name='Sheet2')

    return df_pct, df_cnt


In [ ]:
# 示例：运行参数扫描并导出到 Excel
# 注意：请确保 users, facilities, demand, radius 已按上文准备好
# 建议把输出路径指向与下方绘图读取一致的路径

w_list = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
p_list = [20, 25, 30, 35]  # 可替换为你的候选数量列表
output_path = r'./solution.xlsx'  # 与绘图读取一致
df_pct, df_cnt = run_param_sweep_and_export(users, facilities_all, demand, radius, w_list, p_list, output_path)
display(df_pct.head())
display(df_cnt.head())


In [ ]:
plt.figure(figsize=(14,7)) 

dataframe = pd.read_excel(io='./solution.xlsx', sheet_name="Sheet1")
colors = ["cornflowerblue", "orange", "tomato", 'gray']

dataframe["primary coverage(%)"] = dataframe["primary coverage(%)"].apply(lambda x: x * 100)
dataframe["backup coverage(%)"] = dataframe["backup coverage(%)"].apply(lambda x: x * 100)

w_arr = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]

for index in range(0, len(w_arr)):

    item = w_arr[index]

    for col in [["primary coverage(%)", 'num'], ["backup coverage(%)", 'num']]:
        y_temp = dataframe.loc[dataframe['w'] == item, [col[0]]]
        y = np.array(y_temp).ravel()  # convert 2D to 1D

        x_temp = dataframe.loc[dataframe['w'] == item, [col[1]]]
        x = np.array(x_temp).ravel()  # convert 2D to 1D
            
        # primary coverage
        if col[0] == "primary coverage(%)":
            label = "primary coverage (w=" + str(item) + ")"
            plt.scatter(x, y, marker='D', label=label, c=colors[index])  # point
            plt.plot(x, y, color=colors[index])  # line
        else:
            label = "backup coverage (w=" + str(item) + ")"
            plt.scatter(x, y, marker='s', label=label, c=colors[index])
            plt.plot(x, y, color=colors[index])

font1 = {'family' : 'Times New Roman','weight' : 'normal','size':20}
font2 = {'family' : 'Times New Roman','weight' : 'normal','size':23}
plt.ylabel("Coverage (%)", font2)
plt.xlabel("Number of sound sensors", font2)
plt.tick_params(labelsize=16)
plt.ylim(0, 100)
# plt.grid(axis='y')
plt.legend(loc=2, ncol=2, prop=font1)  # Convert into 2 columns
plt.savefig('coverage_percentage.jpg',dpi=600)
plt.show()



In [ ]:
plt.figure(figsize=(14,7)) 

dataframe = pd.read_excel(io='./solution.xlsx', sheet_name="Sheet2")
colors = ["cornflowerblue", "orange", "tomato", 'gray']


w_arr = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]

for index in range(0, len(w_arr)):

    item = w_arr[index]

    for col in [["primary coverage", 'num'], ['backup coverage', 'num']]:
        y_temp = dataframe.loc[dataframe['w'] == item, [col[0]]]
        y = np.array(y_temp).ravel()  # convert 2D to 1D

        x_temp = dataframe.loc[dataframe['w'] == item, [col[1]]]
        x = np.array(x_temp).ravel()  # convert 2D to 1D
        # primary coverage
        if col[0] == "primary coverage":
            label = "primary coverage (w=" + str(item) + ")"
            plt.scatter(x, y, marker='D', label=label, c=colors[index])  # point
            plt.plot(x, y, color=colors[index])  # line
        else:
            label = "backup coverage (w=" + str(item) + ")"
            plt.scatter(x, y, marker='s', label=label, c=colors[index])
            plt.plot(x, y, color=colors[index])

font1 = {'family' : 'Times New Roman','weight' : 'normal','size':20}
font2 = {'family' : 'Times New Roman','weight' : 'normal','size':23}
plt.ylabel("Demand points covered", font2)
plt.xlabel("Number of sound sensors", font2)
plt.tick_params(labelsize=16)
ymax = float(max(dataframe['primary coverage'].max(), dataframe['backup coverage'].max()))
plt.ylim(0, max(10, ymax * 1.1))  # 给 10% 余量，最小上限 10
# plt.grid(axis='y')
plt.legend(loc=2, ncol=2, prop=font1)  # Convert into 2 columns
plt.savefig('coverage_points.jpg',dpi=600)
plt.show()

    


In [ ]:
# 1) 基于最近邻距离取半径候选（米）
from sklearn.neighbors import NearestNeighbors
d,_ = NearestNeighbors(n_neighbors=2).fit(users).kneighbors(users)
d1_m = d[:,1] * S
radius_list = [np.percentile(d1_m, q) for q in [25, 50, 75, 90]]  # 或直接 [200, 300, 500, 800]

# 2) 批量跑不同半径（输出带半径标签的文件，或汇总到一个表）
all_pct, all_cnt = [], []
for r_m in radius_list:
    r_norm = r_m / S
    df_pct, df_cnt = run_param_sweep_and_export(
        users, facilities_all, demand, r_norm,
        w_list, p_list,
        output_path=f'./solution_radius_{int(r_m)}m.xlsx'
    )
    df_pct['radius_m'] = r_m; df_cnt['radius_m'] = r_m
    all_pct.append(df_pct); all_cnt.append(df_cnt)

df_pct_all = pd.concat(all_pct, ignore_index=True)
df_cnt_all = pd.concat(all_cnt, ignore_index=True)

In [ ]:
# 3) 边际收益与拐点（以某 ω 为例）
w0 = 0.8
sub = (df_pct_all[df_pct_all['w'].round(2).eq(round(w0,2))]
       .sort_values(['radius_m','num']))
sub['marginal_%pt_per_sensor'] = sub.groupby('radius_m')['primary coverage(%)'].diff() / sub.groupby('radius_m')['num'].diff()
# 可据此绘图与定位边际收益明显下降的 p（拐点）

In [ ]:
# 选择一个权重画图（可改成你想看的 ω）
w0 = 0.8
sub = (df_pct_all[df_pct_all['w'].round(2).eq(round(w0, 2))]
       .sort_values(['radius_m','num']))

import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
for r_m, g in sub.groupby('radius_m'):
    g = g.sort_values('num')
    plt.plot(g['num'], g['primary coverage(%)']*100, marker='o', label=f'primary r={int(r_m)}m')
    plt.plot(g['num'], g['backup coverage(%)']*100, marker='s', linestyle='--', label=f'backup r={int(r_m)}m')
plt.ylabel('Coverage (%)'); plt.xlabel('Number of sound sensors')
plt.ylim(0, 100); plt.legend(ncol=2); plt.tight_layout(); plt.show()

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

df = pd.read_csv('data/电网/powertower_tianhe_a/HaizhuPowertower.csv')
x, y, a = df['POINT_X'].to_numpy(), df['POINT_Y'].to_numpy(), df['a'].to_numpy()

# 为了避免极端值主导色带，可用分位数截断；如不需要可改为 a.min()/a.max()
vmin, vmax = np.percentile(a, 2), np.percentile(a, 98)

fig, ax = plt.subplots(figsize=(14,6), dpi=200)
fig.patch.set_facecolor('white'); ax.set_facecolor('white')

sc = ax.scatter(x, y, c=a, cmap='viridis', s=10, linewidths=0,
                vmin=vmin, vmax=vmax, alpha=0.95)

ax.set_aspect('equal', 'box')
ax.axis('off')  # 不显示横纵轴

cb = plt.colorbar(sc, ax=ax, fraction=0.03, pad=0.02)
cb.set_label('S_i')

plt.tight_layout(pad=0)
plt.savefig('towers_gradient.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

import os, geopandas as gpd
from shapely.geometry import MultiPoint
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

# 1) 叠加海珠区边界
boundary_path = 'data/电网/powertower_tianhe_a/HaizhuPowertower_3857.shp'  # 如有真实边界文件，改为其路径
if os.path.exists(boundary_path):
    gdf = gpd.read_file(boundary_path)
    gdf.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, zorder=2)
else:
    hull = MultiPoint(np.column_stack([x, y])).convex_hull
    gpd.GeoSeries([hull]).plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, zorder=2)

# 2) 添加“点=电塔”的图例项
tower_handle = plt.Line2D([], [], marker='o', linestyle='None',
                          markerfacecolor='k', markeredgecolor='k',
                          markersize=6, label='Power towers')
boundary_handle = mpatches.Patch(facecolor='none', edgecolor='black',
                                 linewidth=1.5, label='Haizhu boundary')

ax.legend(handles=[tower_handle, boundary_handle], loc='lower right', frameon=True)

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

# 若上面已有 df/a，可跳过这两行
# df = pd.read_csv('data/电网/powertower_tianhe_a/HaizhuPowertower.csv')
# a = df['a'].to_numpy()

fig, ax = plt.subplots(figsize=(10,4), dpi=200)
ax.hist(a, bins=40, color='#4C78A8', edgecolor='white')
ax.set_title('Distribution of S_i')
ax.set_xlabel('S_i')
ax.set_ylabel('Count')
ax.grid(alpha=0.2)
plt.tight_layout()
plt.show()

In [ ]:
# 散点按 a 着色 + 海珠区边界 + 图例
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.patches as mpatches

# 读取电塔点
df = pd.read_csv('data/电网/powertower_tianhe_a/HaizhuPowertower.csv')
x, y, a = df['POINT_X'].to_numpy(), df['POINT_Y'].to_numpy(), df['a'].to_numpy()

# 色带上下限（避免极端值主导）
vmin, vmax = np.percentile(a, 2), np.percentile(a, 98)

fig, ax = plt.subplots(figsize=(14,6), dpi=200)
fig.patch.set_facecolor('white'); ax.set_facecolor('white')

# 电塔散点
sc = ax.scatter(x, y, c=a, cmap='viridis', s=10, linewidths=0,
                vmin=vmin, vmax=vmax, alpha=0.95, zorder=3)
ax.set_aspect('equal', 'box')
ax.axis('off')  # 不显示坐标轴

# 叠加海珠区行政边界（使用你提供的路径）
boundary_path = r'D:\Academic\Task116_ApplyScience\power_tower_project\GISProj\行政区\海珠区.shp'
try:
    if os.path.exists(boundary_path):
        gdf = gpd.read_file(boundary_path)
        # 投影到与点一致的坐标（点为投影米制；若边界已是米制，则保持）
        try:
            gdf = gdf.to_crs(epsg=3857)
        except Exception:
            pass
        gdf.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.5, zorder=2)
    else:
        print(f'Boundary file not found: {boundary_path}')
except Exception as e:
    print(f'Boundary overlay skipped: {e}')

# 图例：点=电塔，线=海珠区边界
tower_handle = plt.Line2D([], [], marker='o', linestyle='None',
                          markerfacecolor='k', markeredgecolor='k',
                          markersize=6, label='Power towers')
boundary_handle = mpatches.Patch(facecolor='none', edgecolor='black',
                                 linewidth=1.5, label='Haizhu boundary')
leg = ax.legend(handles=[tower_handle, boundary_handle], loc='lower right', frameon=True)
leg.set_zorder(4)

# 颜色条
cb = plt.colorbar(sc, ax=ax, fraction=0.03, pad=0.02)
cb.set_label('S_i')

plt.tight_layout(pad=0)
plt.savefig('towers_gradient_with_boundary.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
# P-性能曲线（w=0.8）：主覆盖、备用覆盖、目标函数 随 P 变化
w0 = 0.8
p_vals = [30, 35, 40, 45, 50, 55]

primary_rates = []   # 0–1
backup_rates = []    # 0–1
obj_vals = []        # 目标函数值

for p in p_vals:
    res = gurobi_solver_bclp(users, facilities_all, demand, p, radius, w=w0)
    if res[0] is None:  # 无解保护
        primary_rates.append(0.0)
        backup_rates.append(0.0)
        obj_vals.append(0.0)
    else:
        _, _, obj, primary_cov, _, _, backup_cov, _ = res
        primary_rates.append(float(primary_cov))
        backup_rates.append(float(backup_cov))
        obj_vals.append(float(obj))

# 转百分比
import numpy as np
primary_pct = np.array(primary_rates) * 100
backup_pct  = np.array(backup_rates) * 100

# 可视化：双 y 轴
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots(figsize=(12, 6))

l1 = ax1.plot(p_vals, primary_pct, marker='o', color='C0', label='Primary coverage (%)')
l2 = ax1.plot(p_vals, backup_pct,  marker='s', linestyle='--', color='C1', label='Backup coverage (%)')
ax1.set_xlabel('Value of P')
ax1.set_ylabel('Coverage (%)')
ax1.set_ylim(0, 100)
ax1.grid(alpha=0.25)

ax2 = ax1.twinx()
l3 = ax2.plot(p_vals, obj_vals, marker='^', color='C2', label='Objective value')
ax2.set_ylabel('Objective value')

lines = l1 + l2 + l3
labels = [ln.get_label() for ln in lines]
ax1.legend(lines, labels, loc='lower right')
plt.title('P-performance curves')
plt.tight_layout()
plt.show()

# 输出数据表便于核对
import pandas as pd
df_p = pd.DataFrame({
    'P': p_vals,
    'primary_coverage_%': primary_pct,
    'backup_coverage_%': backup_pct,
    'Obj': obj_vals,
})
display(df_p)



In [ ]:
# 批量生成不同 P 的选址结果图（P=25,30,35,40），样式与现有 plot_result 一致
import matplotlib.pyplot as plt
radius = 1000
p_list = [25, 30, 35, 40]
# 归一化半径（若已存在 radius 则直接使用）
radius_norm = (real_radius / S) if 'S' in globals() else radius

for P in p_list:
    x_result, y_result, obj, cov_primary, covered_demand, total_demand, cov_backup, backup_covered_demand = \
        gurobi_solver_bclp(users, facilities_all, demand, P, radius_norm, w=0.8)

    if x_result is None:
        print(f"P={P}: no optimal solution")
        continue

    # 选中的设施索引
    solutions = [i for i, flag in enumerate(x_result) if flag]
    opt_sites = ef_.iloc[solutions]

    # 绘图（标题与样式保持一致）
    ax = plot_result(ls, opt_sites, real_radius)
    ax.set_title(f'Selected {P} Sites that Serve {real_radius} m(Gurobi)', fontsize=20)

    # 保存图片
    out_path = f'selected_sites_P{P}_{int(real_radius)}m.png'
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Saved: {out_path}")


In [ ]:
# Export selected sites' POINT_X, POINT_Y for P in [25, 30, 35, 40]
import pandas as pd

radius = real_radius  # 用于命名
p_list = [25, 30, 35, 40]
omega = 0.8  # 与你当前循环一致
rows = []

for P in p_list:
    x_result, y_result, obj, cov_primary, covered_demand, total_demand, cov_backup, backup_covered_demand = \
        gurobi_solver_bclp(users, facilities_all, demand, P, (real_radius / S) if 'S' in globals() else radius, w=omega)

    if x_result is None:
        print(f"P={P}: no optimal solution")
        continue

    # 选中的设施索引与坐标
    solutions = [i for i, flag in enumerate(x_result) if flag]
    opt_sites = ef_.iloc[solutions]  # ef_ 应包含 POINT_X / POINT_Y

    df = opt_sites[['POINT_X', 'POINT_Y']].copy()
    df.insert(0, 'site_index', opt_sites.index)   # 保留原始索引
    df['P'] = P
    df['w'] = omega
    df['radius_m'] = int(real_radius)

    # 单个 P 的导出
    per_path = f'selected_sites_P{P}_{int(real_radius)}m.csv'
    df.to_csv(per_path, index=False, encoding='utf-8-sig')
    print(f"Saved: {per_path}")

    rows.append(df)

# 汇总导出
if rows:
    all_df = pd.concat(rows, ignore_index=True)
    all_path = f'selected_sites_all_P_{int(real_radius)}m.csv'
    all_df.to_csv(all_path, index=False, encoding='utf-8-sig')
    display(all_df.head())
    print(f"Saved combined: {all_path}")

In [ ]:
# 四宫格展示不同 P 的选址结果（P=25,30,35,40）
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib import colors

# 在给定 ax 上绘制结果，复用现有样式（背景密度 + 圆圈 + 十字点）
def plot_result_on_ax(ls, opt_sites, radius_m, ax):
    cmap = plt.cm.Blues
    new_cmap = colors.ListedColormap(cmap(np.linspace(0.15, 1, 256)))
    ls.plot(ax=ax, column=ls['mianji'], k=5, markersize=8, cmap=new_cmap, label='over_65 Population')

    for i, row in enumerate(opt_sites.itertuples(index=False)):
        x = float(row.POINT_X); y = float(row.POINT_Y)
        ax.scatter(x, y, c='C1', marker='+', s=80, linewidths=1.2, label='Optimized Selected Elder Facilities' if i==0 else None)
        circ = Circle((x, y), radius=radius_m/2, fill=False, edgecolor='C1', linewidth=2)
        ax.add_patch(circ)

    ax.set_aspect('equal')
    ax.tick_params(axis='both', left=False, top=False, right=False, bottom=False,
                   labelleft=False, labeltop=False, labelright=False, labelbottom=False)

# 归一化半径与真实半径
radius_norm = (real_radius / S) if 'S' in globals() else radius
radius_m = real_radius

p_list = [25, 30, 35, 40]
fig, axes = plt.subplots(2, 2, figsize=(20, 12), constrained_layout=True)

for ax, P in zip(axes.ravel(), p_list):
    x_result, y_result, obj, cov_primary, covered_demand, total_demand, cov_backup, backup_covered_demand = \
        gurobi_solver_bclp(users, facilities_all, demand, P, radius_norm, w=0.8)
    if x_result is None:
        ax.set_title(f'P={P}: no solution')
        ax.axis('off')
        continue

    sel_idx = [i for i, flag in enumerate(x_result) if flag]
    opt_sites = ef_.iloc[sel_idx]
    plot_result_on_ax(ls, opt_sites, radius_m, ax)
    ax.set_title(f'P={P}', fontsize=16)

# 总标题与统一图例
fig.suptitle(f'Selected Sites at {radius_m} m (Gurobi)', fontsize=22)
handles, labels = axes[0,0].get_legend_handles_labels()
if handles:
    fig.legend(handles, labels, loc='lower right', frameon=True)

out_path = f'selected_sites_grid_P_{int(radius_m)}m.png'
plt.savefig(out_path, dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {out_path}')


In [ ]:
# 汇总图：每个 ω 一张图，展示主/备覆盖与目标值随 P 的变化（修正：显式使用归一化半径 r_norm=real_radius/S，并在本单元重建归一化坐标）
import numpy as np
import matplotlib.pyplot as plt

# 配置：权重与 P 值范围（可按需修改）
w_list = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
p_vals = [25, 30, 35, 40]

# 始终显式使用归一化半径，防止混用尺度
r_norm = float(real_radius) / float(S)

# 在本单元内重建归一化坐标，避免被其他单元修改
users_norm = np.array(ls[['NORM_X', 'NORM_Y']], dtype=float)
facilities_norm = np.array(ef_[['NORM_X', 'NORM_Y']], dtype=float)
demand_arr = np.array(demand, dtype=float)

for w0 in w_list:
    primary_rates, backup_rates, obj_vals = [], [], []
    for p in p_vals:
        res = gurobi_solver_bclp(users_norm, facilities_norm, demand_arr, p, r_norm, w=w0)
        if res[0] is None:
            primary_rates.append(0.0)
            backup_rates.append(0.0)
            obj_vals.append(0.0)
        else:
            _, _, obj, primary_cov, _, _, backup_cov, _ = res
            primary_rates.append(float(primary_cov))
            backup_rates.append(float(backup_cov))
            obj_vals.append(float(obj))

    primary_pct = np.array(primary_rates) * 100.0
    backup_pct  = np.array(backup_rates)  * 100.0

    fig, ax1 = plt.subplots(figsize=(10, 5))
    l1 = ax1.plot(p_vals, primary_pct, marker='o', color='C0', label='Primary coverage (%)')
    l2 = ax1.plot(p_vals, backup_pct,  marker='s', linestyle='--', color='C1', label='Backup coverage (%)')
    ax1.set_xlabel('P')
    ax1.set_ylabel('Coverage (%)')
    ax1.set_ylim(0, 100)
    ax1.grid(alpha=0.3)

    ax2 = ax1.twinx()
    l3 = ax2.plot(p_vals, obj_vals, marker='^', color='C2', label='Objective value')
    ax2.set_ylabel('Objective value')

    lines = l1 + l2 + l3
    labels = [ln.get_label() for ln in lines]
    ax1.legend(lines, labels, loc='lower right')
    plt.title(f'Coverage and Objective vs P (w={w0:.2f})')
    plt.tight_layout()

    out_path = f'summary_w_{w0:.2f}_P_curve.png'
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.show()
    print(f'Saved: {out_path}')


In [ ]:
# 汇总图（六合一）：2列×3行，分别为 w ∈ {1.0,0.8,0.6,0.4,0.2,0.0}
import numpy as np
import matplotlib.pyplot as plt

# 配置
w_list = [1.0, 0.8, 0.6, 0.4, 0.2, 0.0]
p_vals = [25, 30, 35, 40]

# 显式归一化半径与坐标
r_norm = float(real_radius) / float(S)
users_norm = np.array(ls[['NORM_X', 'NORM_Y']], dtype=float)
facilities_norm = np.array(ef_[['NORM_X', 'NORM_Y']], dtype=float)
demand_arr = np.array(demand, dtype=float)

fig, axes = plt.subplots(3, 2, figsize=(14, 12), constrained_layout=True, sharex=True)

legend_handles, legend_labels = None, None

for idx, w0 in enumerate(w_list):
    ax = axes[idx // 2, idx % 2]
    primary_rates, backup_rates, obj_vals = [], [], []

    for p in p_vals:
        res = gurobi_solver_bclp(users_norm, facilities_norm, demand_arr, p, r_norm, w=w0)
        if res[0] is None:
            primary_rates.append(0.0)
            backup_rates.append(0.0)
            obj_vals.append(0.0)
        else:
            _, _, obj, primary_cov, _, _, backup_cov, _ = res
            primary_rates.append(float(primary_cov))
            backup_rates.append(float(backup_cov))
            obj_vals.append(float(obj))

    primary_pct = np.array(primary_rates) * 100.0
    backup_pct  = np.array(backup_rates)  * 100.0

    l1 = ax.plot(p_vals, primary_pct, marker='o', color='C0', label='Primary coverage (%)')
    l2 = ax.plot(p_vals, backup_pct,  marker='s', linestyle='--', color='C1', label='Backup coverage (%)')
    ax.set_ylabel('Coverage (%)')
    ax.set_ylim(0, 100)
    ax.grid(alpha=0.3)

    ax2 = ax.twinx()
    l3 = ax2.plot(p_vals, obj_vals, marker='^', color='C2', label='Objective value')
    ax2.set_ylabel('Objective value')

    ax.set_title(f'w = {w0:.2f}', fontsize=12)

    # 仅保存一次图例句柄
    if legend_handles is None:
        legend_handles = l1 + l2 + l3
        legend_labels = [ln.get_label() for ln in legend_handles]

for ax in axes[-1, :]:
    ax.set_xlabel('P')

# 底部统一图例
fig.legend(legend_handles, legend_labels, loc='lower center', ncol=3, frameon=True)

out_path = 'summary_w_all_P_curves_grid.png'
plt.savefig(out_path, dpi=300, bbox_inches='tight')
plt.show()
print(f'Saved: {out_path}')


In [ ]:
# 导出指定位置索引的坐标到 CSV
import pandas as pd

# 你的解（按 iloc 位置索引）
selected_indices = [215, 132, 69, 282, 155, 112, 102, 79, 255, 163, 187, 245, 107, 123, 177, 15, 63, 23, 172, 181, 28, 149, 242, 265, 76]

# 优先使用内存中与求解一致的数据框
_df = None
for _name in ("ef_", "ef"):
    if _name in globals():
        _df = globals()[_name]
        break
if _df is None:
    _df = pd.read_csv(r"./data/电网/powertower_tianhe_a/HaizhuPowertower.csv")

coords = _df.iloc[selected_indices][['POINT_X', 'POINT_Y']].copy()
coords.insert(0, 'pos_index', selected_indices)

out_path = 'selected_sites_coords.csv'
coords.to_csv(out_path, index=False, encoding='utf-8-sig')
print('Saved:', out_path)
coords.head()


In [ ]:
# 你的解的下标序列（按 iloc 的位置索引）
idx = [215, 132, 69, 282, 155, 112, 102, 79, 255, 163, 187, 245, 107, 123, 177, 15, 63, 23, 172, 181, 28, 149, 242, 265, 76]

import pandas as pd

# 若内存里已有 ef_（或 ef）与求解时一致，优先使用；否则从CSV读
df = None
for name in ("ef_", "ef"):
    if name in globals():
        df = globals()[name]
        break
if df is None:
    df = pd.read_csv(r"D:\Academic\Task116_ApplyScience\allover\BSLCP\data\电网\powertower_tianhe_a\HaizhuPowertower.csv")

# 按位置取 POINT_X/POINT_Y
coords = df.iloc[idx][['POINT_X', 'POINT_Y']].copy()
coords.insert(0, 'pos_index', idx)  # 保留原位置索引
print(coords.to_string(index=False))

# 可选：导出
coords.to_csv('selected_coords.csv', index=False)